# 舵机控制库使用指南与演示
这是一个交互式指南，用于演示如何使用 servo_controller_library.py 来控制连接到树莓派的舵机。请确保 servo_controller_library.py 文件与此 Notebook 位于同一目录下。



## 导入与初始化
首先，我们需要从库文件中导入所有必需的类，并实例化主控制器 ServoController。然后，我们配置将要使用的舵机。

In [ ]:
import time
from servo_controller_library import ServoController, ServoGroup, Easing

# 实例化主控制器
# 如果您的舵机驱动板I2C地址不是0x2D，请在这里修改
controller = ServoController(board_address=0x2D)

# 检查I2C总线是否连接成功
if controller.bus:
    # 配置1号舵机 (水平)，并获取其控制句柄
    h_servo = controller.setup_servo(1, max_angle=270)
    
    # 配置2号舵机 (垂直)，设定安全范围为10%-90%，并获取其控制句柄
    v_servo = controller.setup_servo(2, max_angle=180, min_safe_percent=10, max_safe_percent=90)
    
    print("\n控制器和舵机初始化成功!")
else:
    print("\n控制器初始化失败，请检查硬件连接和I2C配置。")


## 执行初始化自检
在开始任何具体操作前，运行一次完整的自检序列，以确保所有舵机都已准备就绪。

In [ ]:
if controller.bus:
    # 这个方法会让所有已配置的舵机执行一次 “归中 -> 最小 -> 最大 -> 归中” 的完整动作
    controller.run_initialization_sequence(duration_per_move=1.5)


## 功能演示 1 - 平滑移动 (阻塞)
move_to() 方法会平滑地移动舵机，但在移动完成前会阻塞当前程序的执行。这对于需要按顺序执行的简单任务很有用。

In [ ]:
if controller.bus:
    print("--- 演示平滑移动 (阻塞) ---")
    
    print("a) 水平舵机: 使用默认缓动，平滑移动到 25%")
    h_servo.move_to(25, duration=1.5)
    
    print("b) 水平舵机: 使用线性缓动(匀速)，移动到 75%")
    h_servo.move_to(75, duration=1.5, easing_func=Easing.linear)
    
    h_servo.move_to(50, duration=1) # 回到中间
    print("演示完成!")

## 功能演示 2 - 异步移动与回调
move_to_async() 方法会在后台移动舵机，不会阻塞主程序。它还支持on_complete回调，当动作完成时会自动执行一个函数。

In [ ]:
if controller.bus:
    print("\n--- 演示异步移动与回调 ---")
    
    # 定义一个当移动完成时要执行的函数
    def move_complete_callback():
        print(">>> 回调函数被触发: 垂直舵机移动完成!")
    
    print("a) 垂直舵机: 开始异步移动到 100%，主程序继续执行...")
    v_servo.move_to_async(100, duration=2, on_complete=move_complete_callback)
    
    # 在舵机移动时，主程序可以做其他事，比如检查状态
    while v_servo.is_moving:
        print(f"  - 主程序检查: 垂直舵机正在移动... 位置: {v_servo.current_position:.1f}%")
        time.sleep(0.5)
        
    print("  - 主程序检查: 垂直舵机已停止移动。")
    v_servo.move_to(50, duration=1) # 回到中间
    print("演示完成!")

## 功能演示 3 - 指令队列
您可以预先为舵机“编排”一整套动作，然后让它在后台自动执行。

In [ ]:
if controller.bus:
    print("\n--- 演示指令队列 ---")
    
    print("a) 为水平舵机编排一套'舞蹈'动作 (归中 -> 弹跳到最左 -> 平滑到最右 -> 归中)")
    h_servo.queue_move(50, 1.0)
    h_servo.queue_move(0, 1.5, easing_func=Easing.ease_out_bounce)
    h_servo.queue_move(100, 2.0, easing_func=Easing.ease_in_out_cubic)
    h_servo.queue_move(50, 1.0)
    
    print("b) 启动序列，它将在后台自动执行所有动作。")
    h_thread = h_servo.start_sequence()
    h_thread.join() # 等待整个序列执行完毕
    print("水平舵机'舞蹈'完成!")

## 功能演示 4 - 舵机编组
使用 ServoGroup 可以将多个舵机作为一个整体来控制，实现复杂的协同运动（姿态切换）。

In [ ]:
if controller.bus:
    print("\n--- 演示舵机编组 ---")
    
    print("a) 创建一个云台(gimbal)编组，包含水平和垂直舵机。")
    gimbal = ServoGroup(horizontal=h_servo, vertical=v_servo)

    # 定义几个姿态
    pose_top_left = {'horizontal': 25, 'vertical': 75}
    pose_bottom_right = {'horizontal': 75, 'vertical': 25}

    print("b) 移动到'左上角'姿态...")
    gimbal.move_to_pose(pose_top_left, duration=2).wait_for_move()
    time.sleep(1)

    print("c) 移动到'右下角'姿态...")
    gimbal.move_to_pose(pose_bottom_right, duration=2).wait_for_move()
    print("演示完成!")


## 清理与收尾
在程序结束时，最好将所有舵机移动到一个安全的位置（如归中），并关闭I2C总线连接。

In [ ]:
if controller.bus:
    print("\n--- 执行清理操作 ---")
    
    # 使用编组让所有舵机同时归中
    final_group = ServoGroup(h=h_servo, v=v_servo)
    final_group.move_to_pose({'h': 50, 'v': 50}, duration=1.0).wait_for_move()
    
    # 关闭I2C总线
    controller.cleanup()
    print("程序结束。")